Name: Akshata Bheemasamudra Mallikarjunappa

Student ID: 23164204

**Improvements:**

Task1:

1. I had previously used textblob's blob.classify to classify the test data for each document in test data. Upon learning the classifier can be called directly, I got rid of the blob.classify.

2. After reviewing Rajesh's notebook, I learnt a new concept of adjusting the space width in print statement using  "" .ljust(width) "" which I inititally struggled to do in Post 1.

Task2:

1. Reviewing Rajesh's notebook helped me not to hardcode splitting of training data but instead, to use a ratio of 0.8 while splitting the data into training and testing sets.

2. Manually calculated the accuracy in contrast to using accuracy function from textblob.classifiers from Post1.

3. Considered the 10-fold cross validation to check the accuracy of model for different variants of training and test data.


**Task1**

In [1]:
import nltk
nltk.download('punkt')
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
from sklearn.model_selection import KFold
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def naiveBayesClassifier(trainingSet,testSet):
  cls = NaiveBayesClassifier(trainingSet)

  for i,j in testSet:
   doc = cls.classify(i)

   print(f'Document : ({i},{j})'.ljust(50), f' predicted class: {doc}\n')

   accuracy = cls.accuracy(testSet)

  print(f'Accuracy: {accuracy}')

In [3]:
trainingSet = [("London is the Capital of GB", "GB"), ("Oxford is a city in GB","GB"), ("Dublin is the capital of Ireland", "IE"), ("Limerick is a city in Ireland", "IE")]
testSet = [("University of Limerick","IE"),("University College Dublin","IE"),("Imperial College London","GB"),("University of Oxford","GB"),("Ireland & GB","GB")]
naiveBayesClassifier(trainingSet, testSet)

Document : (University of Limerick,IE)              predicted class: IE

Document : (University College Dublin,IE)           predicted class: IE

Document : (Imperial College London,GB)             predicted class: GB

Document : (University of Oxford,GB)                predicted class: GB

Document : (Ireland & GB,GB)                        predicted class: IE

Accuracy: 0.8


**Task2**

In [4]:

!wget https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

--2023-12-02 14:32:06--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.207, 142.251.16.207, 172.253.62.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘bbc-text.csv.4’

bbc-text.csv.4      100%[===================>]   4.82M  --.-KB/s    in 0.03s   

2023-12-02 14:32:06 (153 MB/s) - ‘bbc-text.csv.4’ saved [5057493/5057493]



In [5]:
df = pd.read_csv('bbc-text.csv', delimiter = ',')
df.head(5)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [6]:
print(df.sample(5))
print('----------------------------------------------------')
print(f'\n Number of documents {len(df)}')
print('----------------------------------------------------')
cat_count = df['category'].value_counts()
print(f'Documents per category\n{cat_count}')
print('----------------------------------------------------')
training_data = df.loc[:int(len(df)*0.80),:]
print(f"train size: {len(training_data)}")
print('----------------------------------------------------')
print(f'training set samples: \n {training_data.sample(5)}')
print('----------------------------------------------------')
testing_data = df.loc[int(len(df)*0.80):,:]
print(f"test size: {len(testing_data)}")
print('----------------------------------------------------')
print(f'training set samples: \n {testing_data.sample(5)}')

           category                                               text
1121           tech  playstation 3 processor unveiled the cell proc...
976           sport  radcliffe proves doubters wrong this won t go ...
1595  entertainment  no charges against tv s cosby us comedian bill...
564           sport  dawson set for new wasps contract european cha...
1463           tech  digital guru floats sub-$100 pc nicholas negro...
----------------------------------------------------

 Number of documents 2225
----------------------------------------------------
Documents per category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64
----------------------------------------------------
train size: 1781
----------------------------------------------------
training set samples: 
       category                                               text
1497  politics  lib dems unveil election slogan the liberal de...
255  

In [7]:
training_data = [(text, category) for text, category in zip(training_data['text'], training_data['category'])]
testing_data = [(text, category) for text, category in zip(testing_data['text'], testing_data['category'])]

In [12]:
classifier = NaiveBayesClassifier(training_data)

predictions = [1 if classifier.classify(text) == category else 0 for text,category in testing_data]
accuracy = sum(predictions)/len(predictions)
print(accuracy)

0.9707865168539326


**10-fold cross-validation**

In [14]:
all_data = training_data + testing_data
all_data = all_data[0:1001]

In [15]:
kfold = KFold(n_splits = 10,shuffle = True, random_state = 42)

In [16]:
accuracies = []
for i, (train_index, test_index) in enumerate(kfold.split(all_data)):

    train_set = [all_data[i] for i in train_index]
    test_set = [all_data[i] for i in test_index]


    classifier = NaiveBayesClassifier(train_set)


    accuracy = sum([1 if classifier.classify(text) == category else 0 for text, category in test_set]) / len(test_set)
    print(f'fold {i} accuracy = {accuracy}')
    accuracies.append(accuracy)

fold 0 accuracy = 0.9504950495049505
fold 1 accuracy = 0.92
fold 2 accuracy = 0.93
fold 3 accuracy = 0.98
fold 4 accuracy = 0.99
fold 5 accuracy = 0.96
fold 6 accuracy = 0.95
fold 7 accuracy = 0.91
fold 8 accuracy = 0.92
fold 9 accuracy = 0.96


In [17]:
average_accuracy = sum(accuracies) / len(accuracies)
print("Average 10-fold cross-validation accuracy:", average_accuracy)

Average 10-fold cross-validation accuracy: 0.9470495049504951


In [22]:


# for i, (train_index, test_index) in enumerate(kfold.split(df['text'],df['category'])):
#    print(f"Fold {i}:")
#    print(f"  Train: index={train_index}")
#    print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[   0    1    2 ... 2222 2223 2224]
  Test:  index=[  18   25   29   44   51   67   69   70   73   96  100  109  111  128
  134  135  168  179  184  192  196  218  231  233  240  247  254  275
  290  297  298  303  307  321  324  332  350  353  360  361  366  368
  374  382  383  402  406  409  414  416  420  422  423  426  438  478
  480  482  514  519  527  528  529  535  544  548  552  554  555  564
  571  582  585  602  613  617  618  629  636  637  643  650  651  677
  678  679  686  700  742  771  780  781  787  812  831  845  849  856
  862  869  892  905  929  937  941  942  962  963  973  976  986  990
 1004 1040 1041 1043 1057 1064 1068 1085 1100 1113 1128 1132 1151 1175
 1177 1181 1185 1193 1204 1232 1233 1264 1269 1270 1287 1317 1320 1341
 1349 1359 1366 1368 1375 1379 1381 1386 1404 1405 1414 1419 1423 1433
 1435 1449 1468 1480 1481 1491 1501 1506 1510 1514 1533 1544 1545 1551
 1552 1582 1618 1622 1644 1702 1710 1716 1730 1737 1746 1748 1756 1763
 17

In [ ]:
# for train,test in kfold.split(df):
#   print('train: %s, test: %s' % (df['text'], df['category']))